In [2]:
#Google Colab and GPU Set UP
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf

device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
!pip install pytorch_pretrained_bert
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertModel, BertForMaskedLM
import nltk
from nltk import sent_tokenize
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import os
import json
nltk.download('punkt')
print(torch.__version__)
print(torch.cuda.memory_allocated())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device,n_gpu)




Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Found GPU at: /device:GPU:0
     |████████████████████████████████| 133kB 5.0MB/s 
     |████████████████████████████████| 645kB 59.2MB/s 


Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
1.3.1+cu100
0
cuda 1


In [0]:
#Setting File Path
base_dir = "drive/My Drive/" #Loading google drive to work on colaboratory
test_file = "sampled_simplified_nq_30000_test_6000.json" #Add your file name here

In [4]:
filename = base_dir + test_file
with open(filename) as json_file:
    data = json.load(json_file)
print(len(data))
print(data[0].keys())

6000
dict_keys(['document_text', 'long_answer_candidates', 'question_text', 'annotations', 'document_url', 'example_id'])


In [5]:
def format_data_for_LM(data):
  ''' Takes the raw data and converts into q_a format i.e. each example has a question + answer attribute '''
  qa_data = []
  c = 0
  for i in range(len(tqdm(data))):
    question = data[i]['question_text']
    doc_text = data[i]['document_text']
    split_text = doc_text.split(' ')
    la_start = data[i]['annotations'][0]['long_answer']['start_token']
    la_end = data[i]['annotations'][0]['long_answer']['end_token']
    long_answer = ' '.join([item for item in split_text[la_start:la_end] if not ('<' in item or '>' in item) ])
    
    q_a = question+" ? "+long_answer
    long_answer_candidate_texts = []
    temp = {}
    temp["answer_index"] = -1
    for long_an_can in data[i]['long_answer_candidates']:
      lac_start = long_an_can['start_token']
      lac_end = long_an_can['end_token']
      if lac_start == la_start and lac_end == la_end:
        temp["answer_index"] = data[i]['long_answer_candidates'].index(long_an_can)

      long_answer_candidate_text = ' '.join([item for item in split_text[lac_start:lac_end] if not ('<' in item or '>' in item) ])
      long_answer_candidate_texts.append(long_answer_candidate_text)
    
    temp['example_id'] = data[i]['example_id']
    temp['q_a'] = q_a
    temp['question'] = question + " ? "
    temp['long_answers'] = long_answer_candidate_texts
    if long_answer == '':
      temp["answer_present"] = False
    else:
      temp["answer_present"] = True
    qa_data.append(temp)
  return qa_data
qa_data = format_data_for_LM(data)

# print(long_answer)

  0%|          | 0/6000 [00:00<?, ?it/s]


In [6]:
print(len(qa_data))
print(qa_data[0].keys())
print(qa_data[11]['answer_index'])
print(qa_data[11]['answer_present'])

6000
dict_keys(['answer_index', 'example_id', 'q_a', 'question', 'long_answers', 'answer_present'])
39
True


In [0]:
#Might save it as well as colab is fragile
# base_dir = "drive/My Drive/"
# data_file = "qa_data.json"
# import json
# filename = base_dir + data_file
# with open(filename,"w") as json_file:
#     json.dump(qa_data,json_file)

In [0]:
#Get qa_data from the file
# base_dir = "drive/My Drive/"
# data_file = "qa_data.json"
# filename = base_dir + data_file
# print(filename)
# import json
# with open(filename) as json_file:
#     qa_data = json.load(json_file)

In [9]:
def filter_qa_data(qa_data):
  filtered_qa_data = [item for item in qa_data if item['answer_present']]
  return filtered_qa_data

filtered_qa_data = filter_qa_data(qa_data)
print(len(filtered_qa_data))
print(filtered_qa_data[0].keys())
print(len(filtered_qa_data[0]['long_answers']))

3009
dict_keys(['answer_index', 'example_id', 'q_a', 'question', 'long_answers', 'answer_present'])
57


In [10]:
#Load the model
# del model
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.cuda()
# file_path = base_dir + "dlt_bertLM_10000_q_a_paragraphs_model_3.pth"
# model.load_state_dict(torch.load(file_path))
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 1182473.04B/s]


In [0]:
def create_data_loader(tokenizer,data):

  sentences = []
  for data_point in data:
    sentences += [data_point['question'] + la for la in data_point['long_answers']]
  tok_sentences = sentences # q_a passed as one sequence
  sentences_with_special_tokens = ["[CLS] " + sentence + " [SEP]" for sentence in tok_sentences]

  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences_with_special_tokens]
  print("Tokenization Done......")
  MAX_LEN = 512
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  inputs = torch.tensor(input_ids)
  print("Padding Done......")
  # Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
  batch_size = 16

  # Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
  # with an iterator the entire dataset does not need to be loaded into memory

  tensor_data = TensorDataset(inputs)
  # train_sampler = RandomSampler(train_data)
  print("Calling Dataloader")
  dataloader = DataLoader(tensor_data, batch_size=batch_size, drop_last=False)
  return dataloader,sentences_with_special_tokens

dataloader,sentences_with_special_tokens = create_data_loader(tokenizer,filtered_qa_data)






In [12]:
# for item in sentences_with_special_tokens[:100]:
#   print(item)
# print(len(dataloader))
# print(scores)

24736


NameError: ignored

In [13]:
output = []
temp = None
scores = []
for step, batch in enumerate(tqdm(dataloader, position=0, leave=True)):
# Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids = batch[0]
  # print(b_input_ids[0][-50:])
  # print(sentences_with_special_tokens[0])
  # print("Scores length is ",len(scores))
  # print(b_input_ids)
  # print("Input id shape",b_input_ids.shape)
  # print("HERE:",b_input_ids, b_input_mask)
  # Telling the model not to compute or store gradients, saving memory and speeding up validation
  # print(b_input_ids)
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    output = model(b_input_ids).cpu().numpy()
    b_input_ids = b_input_ids.cpu().numpy()
    # print(output.shape)
    # print(b_input_ids.shape)
    
    for i in range(len(b_input_ids)):
      c = 0
      temp = []
      for item in b_input_ids[i]:
        if item:
          temp.append(output[i,c,item])
        c += 1
      # print(len(temp),temp)
      scores.append(np.sum(np.array(temp))/len(temp))


    # print(len(output),output)

100%|██████████| 24736/24736 [4:01:38<00:00,  1.76it/s]


In [14]:
print(len(scores))
# print(scores[0:50])

395773


In [16]:
print(filtered_qa_data[0].keys())
start = 0
end = 0
for i in range(len(filtered_qa_data)):
  # print("Number of long answers are {}".format(len(filtered_qa_data[i]['long_answers'])))
  end += len(filtered_qa_data[i]['long_answers'])
  scr = scores[start:end]
  filtered_qa_data[i]['scores'] = scr
  start = end



dict_keys(['answer_index', 'example_id', 'q_a', 'question', 'long_answers', 'answer_present'])


In [0]:
with open("filtered_qa_6000.json","w") as f:
  json.dump(filtered_qa_data,f)

#The rest of the windows are just experiments.

In [0]:
for item in filtered_qa_data:
  # print(len(item['long_answers']))
  if len(item['long_answers']) != len(item['scores']):
    print("LOL")


In [40]:
print(len(filtered_qa_data[1]['long_answers']),len(filtered_qa_data[1]['scores']))
print(filtered_qa_data[1]['answer_index'])
print(filtered_qa_data[1]['scores'][filtered_qa_data[1]['answer_index']])
print(np.argsort(np.array(filtered_qa_data[1]['scores'])))
print(filtered_qa_data[1].keys())
print(np.min(np.array(filtered_qa_data[1]['scores'])), np.max(np.array(filtered_qa_data[1]['scores'])),np.median(np.array(filtered_qa_data[1]['scores'])))

143 143
93
18.509088689630683
[ 94  19  16   4   1   7  65 121 127 114 128 104 118 123  99   3  96  97
 103 102  64 126 124  67 111 125   8 120 113  20 117 116 110 122  98 115
 134  63 119 107 130 135   2 131 100  26 112 129 106 133 105 137 108 109
  25 101  18  66  73   5 140  82 138 132  95 136  11  12  13   9 139  71
  33  59  61  29  21  34  62  17  48   6  22  24  23 141  37  36  39  41
  28  44  43   0  72  47  32  76  46  68  15  14  80  56  53  38  78  10
  52  45  58 142  60  54  77  51  93  40  70  49  86  91  75  69  31  87
  92  79  85  57  90  30  50  55  74  88  27  42  84  89  81  83  35]
dict_keys(['answer_index', 'example_id', 'q_a', 'question', 'long_answers', 'answer_present', 'scores'])
9.367835998535156 20.17344857865021 15.935251596811655


In [60]:
def top_k_precision(filtered_qa_data,k = 5, threshold = 15.00):
  t = 0
  p = 0
  p_th = 0
  count = 0
  av_answer_length = 0
  high = 0
  low = 10000000
  scrs = []
  for sample in filtered_qa_data:
    if sample['answer_index'] in np.flip(np.argsort(np.array(sample['scores'])))[:k]:
      p += 1
    t += 1
    av_answer_length += len(sample['scores'])
    if len(sample['scores']) > high:
      high = len(sample['scores'])
    if len(sample['scores']) < low:
      low = len(sample['scores'])
    scrs += sample['scores']
    if sample['answer_index'] in np.argwhere(np.array(sample['scores']) >= threshold ).flatten():
      p_th += 1
    count += len(np.argwhere(np.array(sample['scores']) >= threshold ).flatten())


  av_answer_length /= len(filtered_qa_data)
  return float(p/t),av_answer_length,high,low,scrs, (p_th/t), (count/t)

precision, av_answer_length,high_la_can,low_la_can,scrs, precision_threhold,av_la_picked = top_k_precision(filtered_qa_data,50,15)
# output = top_k_precision(filtered_qa_data,50,15)
print(precision, av_answer_length,high_la_can,low_la_can,scrs, precision_threhold,av_la_picked)

print("Min,Max,Mean and Median of scores are...")
print(np.min(np.array(scrs)),np.max(np.array(scrs)),np.mean(np.array(scrs)),np.median(np.array(scrs)))
print(len(filtered_qa_data))



0.8368228647391159 131.52974410103025 1885 1 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
question_length = len(tokenizer.tokenize(question))
c_q_a = []
c_q = []
c_a = []
q_sum = []
a_sum = []
q_a_sum = []
for k in range(len(input_ids)):
  temp_c_q_a = 0
  temp_c_q = 0
  temp_c_a = 0
  temp_q_sum = 0.0
  temp_a_sum = 0.0
  temp_q_a_sum = 0.0
  for i in range(1,len(input_ids[k])-1):
    if (input_ids[k][i+1] != 0):
      temp_c_q_a += 1
      temp_q_a_sum += output[k][:,i,input_ids[k][i]][0].cpu().numpy()
    if (i < 1+question_length ):
      temp_c_q += 1
      temp_q_sum += output[k][:,i,input_ids[k][i]][0].cpu().numpy()
    elif (i >= 1+question_length and input_ids[k][i+1] != 0):
      temp_c_a += 1
      temp_a_sum += output[k][:,i,input_ids[k][i]][0].cpu().numpy()
  if input_ids[k][-1] != 0:
    temp_a_sum += output[k][:,i,input_ids[k][-1]][0].cpu().numpy()
    temp_q_a_sum += output[k][:,i,input_ids[k][-1]][0].cpu().numpy()

  c_q_a.append(temp_c_q_a)
  c_q.append(temp_c_q)
  c_a.append(temp_c_a)
  q_sum.append(temp_q_sum)
  a_sum.append(temp_a_sum)
  q_a_sum.append(temp_q_a_sum)
  
  



In [0]:
print("Question......")
print(q_sum)
print(c_q)
normalized_q_sum = np.array(q_sum)/np.array(c_q)
print(np.argmin(q_sum),np.argmax(q_sum))
print(np.argmin(normalized_q_sum),np.argmax(normalized_q_sum))
print("Answers......")
print(a_sum)
print(c_a)
normalized_a_sum = np.array(a_sum)/np.array(c_a)
print(np.argmin(a_sum),np.argmax(a_sum))
print(np.argmin(normalized_a_sum),np.argmax(normalized_a_sum))
print("Question+Answers.......")
print(q_a_sum)
print(c_q_a)
normalized_q_a_sum = np.array(q_a_sum)/np.array(c_q_a)
print(np.argmin(q_a_sum),np.argmax(q_a_sum))
print(np.argmin(normalized_q_a_sum),np.argmax(normalized_q_a_sum))

In [0]:
arg_sort = np.argsort(q_a_sum)
# print(q_a_sum[36])
# print(arg_sort)
predictions = []
scores = []
for i in range(len(q_a_sum)-1,-1,-1):
  # print(i,arg_sort[i])
  if arg_sort[i] == 0:
    predictions.append(sentences[arg_sort[i]])
    scores.append(q_a_sum[arg_sort[i]])
    break
  else:
    predictions.append(sentences[arg_sort[i]])
    scores.append(q_a_sum[arg_sort[i]])

for i in range(len(predictions)):
  print(scores[i],predictions[i])
# print(sentences[0])
# print(sentences[16])
# print(sentences[0])
# print(q_a_sum[0],q_a_sum[16])
# print(input_ids[0] == input_ids[16])
# print(input_ids[16])

In [0]:
#Load the model
# del model
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.cuda()
file_path = base_dir + "dlt_bertLM_10000_q_a_paragraphs_model_3.pth"
model.load_state_dict(torch.load(file_path))
model.eval()


In [0]:
ind = 8
temp = qa_data[:100]
top_score_ft = []
correct_answer_score_ft = []
position_ft = []

for ind in range(len(temp)):
  try:
    # print("Testing Cuda Memory Usage",torch.cuda.memory_allocated())
    q_a = temp[ind]['q_a']
    question = temp[ind]['question']
    sentences = []
    sentences.append(q_a)
    for item in temp[ind]['long_answers']:
      sentences.append(question+item)

    # tok_sentences = sent_tokenize(sentences)
    tok_sentences = sentences # q_a passed as one sequence
    # print(tok_sentences)
    sentences_with_special_tokens = ["[CLS] " + sentence + " [SEP]" for sentence in tok_sentences]


    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences_with_special_tokens]
    MAX_LEN = 256
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    train_inputs = torch.tensor(input_ids)


    # Create attention masks
    attention_masks = []


    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids:
      seq_mask = [float(i>0) for i in seq]
      attention_masks.append(seq_mask)

    train_masks = torch.tensor(attention_masks)



    # Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
    batch_size = min(16,input_ids.shape[0])

    # Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
    # with an iterator the entire dataset does not need to be loaded into memory

    train_data = TensorDataset(train_inputs, train_masks)
    # train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, batch_size=batch_size, drop_last=False)
    # print("Testing Cuda Memory Usage",torch.cuda.memory_allocated())

    #Load the model
    # del model
    # model = BertForMaskedLM.from_pretrained('bert-base-uncased')
    # model.cuda()
    # file_path = base_dir + "dlt_bertLM_10000_q_a_paragraphs_model_3.pth"
    # model.load_state_dict(torch.load(file_path))
    # model.eval()

    # print("Testing Cuda Memory Usage",torch.cuda.memory_allocated())
    #Run Evaluation
    output = []
    for step, batch in enumerate(tqdm(train_dataloader, position=0, leave=True)):
      # print("inside eval")
    # Add batch to GPU
      batch = tuple(t.to(device) for t in batch)
      # Unpack the inputs from our dataloader
      b_input_ids, b_input_mask = batch
      # Telling the model not to compute or store gradients, saving memory and speeding up validation
      with torch.no_grad():
        # Forward pass, calculate logit predictions
        temp_o = model(b_input_ids, attention_mask=b_input_mask)
        for i in range(len(temp_o)):
          output.append(temp_o[i])
        del temp_o

    # print("Output shape",len(output),output[0].shape,output[0][5,7].cpu().numpy())
    with torch.no_grad():
      question_length = len(tokenizer.tokenize(question))
      c_q_a = []
      c_q = []
      c_a = []
      q_sum = []
      a_sum = []
      q_a_sum = []
      for k in range(len(input_ids)):
        temp_c_q_a = 0
        temp_c_q = 0
        temp_c_a = 0
        temp_q_sum = 0.0
        temp_a_sum = 0.0
        temp_q_a_sum = 0.0
        for i in range(1,len(input_ids[k])-1):
          if (input_ids[k][i+1] != 0):
            temp_c_q_a += 1
            temp_q_a_sum += output[k][i,input_ids[k][i]].cpu().numpy()
          if (i < 1+question_length ):
            temp_c_q += 1
            temp_q_sum += output[k][i,input_ids[k][i]].cpu().numpy()
          elif (i >= 1+question_length and input_ids[k][i+1] != 0):
            temp_c_a += 1
            temp_a_sum += output[k][i,input_ids[k][i]].cpu().numpy()
        if input_ids[k][-1] != 0:
          temp_a_sum += output[k][i,input_ids[k][-1]].cpu().numpy()
          temp_q_a_sum += output[k][i,input_ids[k][-1]].cpu().numpy()
        
        c_q_a.append(temp_c_q_a)
        c_q.append(temp_c_q)
        c_a.append(temp_c_a)
        q_sum.append(temp_q_sum)
        a_sum.append(temp_a_sum)
        q_a_sum.append(temp_q_a_sum)
      del output
      # print("Question......")
      # print(question)
      # print("Actual Answer......")
      # print(q_a)
      normalized_q_sum = np.array(q_sum)/np.array(c_q)
      normalized_a_sum = np.array(a_sum)/np.array(c_a)
      normalized_q_a_sum = np.array(q_a_sum)/np.array(c_q_a)
      per_difference = (normalized_q_a_sum-normalized_a_sum)*100/normalized_a_sum
      # print(np.argsort(per_difference))
      arg_sort = np.argsort(normalized_q_a_sum)
      # arg_sort = np.argsort(per_difference)
      predictions = []
      scores = []
      for i in range(len(normalized_q_a_sum)-1,-1,-1):
        if arg_sort[i] == 0:
          predictions.append(sentences[arg_sort[i]])
          scores.append(normalized_q_a_sum[arg_sort[i]])
          break
        else:
          predictions.append(sentences[arg_sort[i]])
          scores.append(normalized_q_a_sum[arg_sort[i]])

      # for i in range(len(predictions)):
      #   print(scores[i],predictions[i])
      top_score_ft.append(scores[0])
      correct_answer_score_ft.append(scores[-1])
      position_ft.append(len(scores))

      print("Testing Cuda Memory Usage",torch.cuda.memory_allocated())
  except:
    pass

  






In [0]:
print(top_score)
print(correct_answer_score)
print(position)
print(np.mean(np.array(position)))

In [0]:
print(top_score_ft)
print(correct_answer_score_ft)
print(position_ft)
print(np.mean(np.array(position_ft)))